In [7]:
# import everything ml
import torch
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import re
import string
from wordcloud import WordCloud
import nltk
from collections import Counter
import contractions
from tqdm import tqdm
import math
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import os
from PIL import Image, ImageFile
from PIL import ImageDraw
import datasets
from datasets import load_dataset

In [19]:
source_path="./data/decodinglab/placedata.npy"
# --- 3. Load and Process Data ---
print(f"\nAttempting to load data from: {source_path}")

try:
    # Load the .npy file.
    # allow_pickle=True is necessary for loading object arrays, like dictionaries.
    # .item() is used to extract the dictionary object that was saved into the file.
    loaded_data = np.load(source_path, allow_pickle=True).item()
    print("File loaded successfully.")

    # --- 4. Extract Variables from the Dictionary ---
    # We extract each variable by using its name as a key in the loaded dictionary.
    # Note that we are now using 'RLtesttgt' as the key.
    LRtrain = loaded_data['LRtrain']
    LRtest = loaded_data['LRtest']
    RLtest = loaded_data['RLtest']
    LRtraintgt = loaded_data['LRtraintgt']
    LRtesttgt = loaded_data['LRtesttgt']
    RLtesttgt = loaded_data['RLtesttgt'] # <-- Using the new, renamed variable

    print("\nSuccessfully extracted all variables. Shapes:")
    print(f"  LRtrain:    {LRtrain.shape}")
    print(f"  LRtest:     {LRtest.shape}")
    print(f"  RLtest:     {RLtest.shape}")
    print(f"  LRtraintgt: {LRtraintgt.shape}")
    print(f"  LRtesttgt:  {LRtesttgt.shape}")
    print(f"  RLtesttgt:  {RLtesttgt.shape}") # <-- Updated variable name

    # --- 5. Find and Print Unique Labels ---
    # The np.unique() function returns the sorted unique elements of an array.
    print("\n" + "="*40)
    print("Analyzing unique labels in target vectors...")
    print("="*40)

    unique_train_labels = np.unique(LRtraintgt)
    print(f"Unique labels in LRtraintgt: {unique_train_labels}")

    unique_test_lr_labels = np.unique(LRtesttgt)
    print(f"Unique labels in LRtesttgt:  {unique_test_lr_labels}")

    unique_test_rl_labels = np.unique(RLtesttgt) # <-- Using the new, renamed variable
    print(f"Unique labels in RLtesttgt:   {unique_test_rl_labels}") # <-- Updated label
    print("="*40)


# --- Error Handling ---
except FileNotFoundError:
    print(f"\n[ERROR] File Not Found: The file could not be found at '{source_path}'.")
    print("Please ensure that 'placedata_modified.npy' exists in the root of your 'My Drive'.")

except KeyError as e:
    print(f"\n[ERROR] Missing Data: The file was loaded, but a required variable key was not found: {e}.")
    print("Please check the script that created 'placedata_modified.npy' to ensure all 6 variables were saved correctly with the new names.")

except Exception as e:
    print(f"\n[ERROR] An unexpected error occurred: {e}")


Attempting to load data from: ./data/decodinglab/placedata.npy
File loaded successfully.

Successfully extracted all variables. Shapes:
  LRtrain:    (359, 916)
  LRtest:     (359, 904)
  RLtest:     (359, 1003)
  LRtraintgt: (916,)
  LRtesttgt:  (904,)
  RLtesttgt:  (1003,)

Analyzing unique labels in target vectors...
Unique labels in LRtraintgt: [0.         0.04545455 0.09090909 0.13636364 0.18181818 0.22727273
 0.27272727 0.31818182 0.36363636 0.40909091 0.45454545 0.5
 0.54545455 0.59090909 0.63636364 0.68181818 0.72727273 0.77272727
 0.81818182 0.86363636 0.90909091 0.95454545 1.        ]
Unique labels in LRtesttgt:  [0.         0.04545455 0.09090909 0.13636364 0.18181818 0.22727273
 0.27272727 0.31818182 0.36363636 0.40909091 0.45454545 0.5
 0.54545455 0.59090909 0.63636364 0.68181818 0.72727273 0.77272727
 0.81818182 0.86363636 0.90909091 0.95454545 1.        ]
Unique labels in RLtesttgt:   [0.         0.04545455 0.09090909 0.13636364 0.18181818 0.22727273
 0.27272727 0.318181

In [20]:
# class PlacedataDataset(torch.utils.data.Dataset):
#     def __init__(self, features, targets):
#         self.features = features
#         self.targets = targets

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, index):
#         feature = torch.from_numpy(self.features[index]) 
#         target = torch.tensor(self.targets[index], dtype=torch.float32)
#         # print(f"feature: {feature}\ntarget: {target}")
#         return feature, target
    
# # For training data
# train_dataset = PlacedataDataset(LRtrain, LRtraintgt)
# # print(LRtraintgt)

# # For test data (left-to-right)
# test_lr_dataset = PlacedataDataset(LRtest, LRtesttgt)

# # For test data (right-to-left)
# test_rl_dataset = PlacedataDataset(RLtest, RLtesttgt)
# train_dataset = PlacedataDataset(LRtrain.T, LRtraintgt)
# test_lr_dataset = PlacedataDataset(LRtest.T, LRtesttgt)
# test_rl_dataset = PlacedataDataset(RLtest.T, RLtesttgt)


# batch_size = 32  # Adjust as needed
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
# test_lr_loader = DataLoader(test_lr_dataset, batch_size=batch_size, shuffle=False)
# test_rl_loader = DataLoader(test_rl_dataset, batch_size=batch_size, shuffle=False)



In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

# --- Parameters ---
source_path = "./data/decodinglab/placedata.npy"
num_inputs = 359
num_classes = 23
learning_rate = 0.01
num_epochs = 50
batch_size = 32

# --- Helper Functions ---
def label_to_class(label_array, num_classes=23):
    scaled = np.floor(label_array * num_classes).astype(int)
    # Ensure maximum label 1.0 maps to class 22 (not 23, which is out of bounds)
    scaled[scaled == num_classes] = num_classes - 1
    return scaled

# --- Load the data ---
loaded_data = np.load(source_path, allow_pickle=True).item()
LRtrain = loaded_data['LRtrain']
LRtest = loaded_data['LRtest']
LRtraintgt = loaded_data['LRtraintgt']
LRtesttgt = loaded_data['LRtesttgt']

# --- Convert targets to class indices ---
y_train_cls = label_to_class(LRtraintgt, num_classes)
y_test_cls = label_to_class(LRtesttgt, num_classes)

# --- Convert to PyTorch tensors ---
X_train = torch.tensor(LRtrain.T, dtype=torch.float32)
X_test = torch.tensor(LRtest.T, dtype=torch.float32)
y_train = torch.tensor(y_train_cls.T, dtype=torch.long)
y_test = torch.tensor(y_test_cls.T, dtype=torch.long)

# --- Define a Single Layer Perceptron ---
class SLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SLP, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

model = SLP(num_inputs, num_classes)

# --- Loss and Optimizer ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# --- Training Loop ---
for epoch in range(num_epochs):
    model.train()
    permutation = torch.randperm(X_train.size(0))

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], y_train[indices]

        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}")

# --- Evaluation ---
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predicted_classes = torch.argmax(outputs, dim=1)
    accuracy = accuracy_score(y_test.numpy(), predicted_classes.numpy())

print(f"\nFinal Test Accuracy: {accuracy * 100:.2f}%")

# --- Optional: Convert Predicted Class Indices back to Center of Position Bin ---
def class_to_position(class_indices, num_classes=23):
    bin_width = 1.0 / num_classes
    return class_indices * bin_width + bin_width / 2  # center of each bin

predicted_positions = class_to_position(predicted_classes.numpy())

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
